In [9]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [10]:
df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=['Tanggal']) #parse = untuk ngubah kolom Tanggal jadi datetime
df = df.sort_values('Tanggal')
df = df.reset_index(drop=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   object        
 2   Temperatur Maksimum                 4020 non-null   object        
 3   Temperatur Rata-rata                4020 non-null   object        
 4   Kelembapan Rata-rata                4020 non-null   object        
 5   Curah Hujan (mm)                    4020 non-null   object        
 6   Lamanya Penyinaran Matahari         4020 non-null   object        
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [12]:
checkstripvalue = (df['Curah Hujan (mm)'] == "-").sum()
print(checkstripvalue)

497


In [13]:
df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)

df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].astype(float)

C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_100\2154469297.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-',0)


In [14]:
checkstripvalue2 = (df['Curah Hujan (mm)'] == 8888).sum()
print(checkstripvalue2)

209


# Fill Missing Values using RF Imputer

In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

df2 = pd.read_excel(
    "Updated_Data_Historis_2015_2025.xlsx",
    parse_dates=["Tanggal"]
)
df2 = df2.sort_values("Tanggal").reset_index(drop=True)

invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ", "8888", 8888]
df2 = df2.replace(invalid_values, np.nan)

# Tangkap SEMUA blank aneh dari Excel
df2 = df2.replace(r'^\s*$', np.nan, regex=True)

col = "Arah Angin Terbanyak (°)"

df2[col] = (
    df2[col].astype(str)
            .str.strip()
            .str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False)
)

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}

df2[col] = df2[col].map(mapping)

# for c in df2.columns:
#     if c != "Tanggal":
#         df2[c] = pd.to_numeric(df2[c], errors="coerce")

# target = "Curah Hujan (mm)"

# feature_cols = [
#     c for c in df2.columns
#     if c not in ["Tanggal", target]
# ]
# df2[feature_cols] = df2[feature_cols].fillna(
#     df2[feature_cols].median()
# )

# train_df = df2[df2[target].notna()]
# pred_df  = df2[df2[target].isna()]

# X_train = train_df[feature_cols]
# y_train = train_df[target]

# X_pred = pred_df[feature_cols]

# rf = RandomForestRegressor(
#     n_estimators=500,
#     random_state=42,
#     n_jobs=-1
# )
# rf.fit(X_train, y_train)

# df2.loc[df2[target].isna(), target] = rf.predict(X_pred)

# df2.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
# print("File disimpan: Ready_To_Use_Updated_RF.xlsx")


C:\Users\Lenovo E15\AppData\Local\Temp\ipykernel_100\1063207960.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2 = df2.replace(invalid_values, np.nan)


In [17]:
# 1. Pastikan hanya mengambil kolom numerik
numeric_df = df2.select_dtypes(include=[np.number])

# 2. Hitung korelasi terhadap kolom target saja
target_corr = numeric_df.corr()['Curah Hujan (mm)'].sort_values(ascending=False)

print(target_corr)

Curah Hujan (mm)                      1.000000
Kelembapan Rata-rata                  0.399640
Arah Angin Saat Kecepatan Maksimum    0.085265
Arah Angin Terbanyak (°)              0.002441
Kecepatan Angin Maksimum             -0.009466
Kecepatan Angin Rata-rata            -0.032244
Lamanya Penyinaran Matahari          -0.199244
Temperatur Maksimum                  -0.319690
Temperatur Rata-rata                 -0.388173
Temperatur Minimum                   -0.402973
Name: Curah Hujan (mm), dtype: float64


In [18]:
for c in df2.columns:
    if c != "Tanggal":
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

target = "Curah Hujan (mm)"

feature_cols = [
    c for c in df2.columns
    if c not in ["Tanggal", target]
]
df2[feature_cols] = df2[feature_cols].fillna(
    df2[feature_cols].median()
)


train_df = df2[df2[target].notna()]
pred_df  = df2[df2[target].isna()]

X_train = train_df[feature_cols]
y_train = train_df[target]

X_pred = pred_df[feature_cols]

rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

df2.loc[df2[target].isna(), target] = rf.predict(X_pred)

df2.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
print("File disimpan: Ready_To_Use_Updated_RF.xlsx")

File disimpan: Ready_To_Use_Updated_RF.xlsx


In [19]:
data1 = pd.read_excel('Ready_To_Use_Updated_RF.xlsx')

In [21]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4020 non-null   datetime64[ns]
 1   Temperatur Minimum                  4020 non-null   float64       
 2   Temperatur Maksimum                 4020 non-null   float64       
 3   Temperatur Rata-rata                4020 non-null   float64       
 4   Kelembapan Rata-rata                4020 non-null   int64         
 5   Curah Hujan (mm)                    4020 non-null   float64       
 6   Lamanya Penyinaran Matahari         4020 non-null   float64       
 7   Kecepatan Angin Maksimum            4020 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4020 non-null   int64         
 9   Kecepatan Angin Rata-rata           4020 non-null   int64         
 10  Arah Angin Terbanyak (°)

In [22]:
data1.isna().sum().sort_values(ascending=False)


Tanggal                               0
Temperatur Minimum                    0
Temperatur Maksimum                   0
Temperatur Rata-rata                  0
Kelembapan Rata-rata                  0
Curah Hujan (mm)                      0
Lamanya Penyinaran Matahari           0
Kecepatan Angin Maksimum              0
Arah Angin Saat Kecepatan Maksimum    0
Kecepatan Angin Rata-rata             0
Arah Angin Terbanyak (°)              0
dtype: int64

# Feature Engineering

In [23]:
df3 = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")

# Pastikan datetime & sort
df3['Tanggal'] = pd.to_datetime(df3['Tanggal'])
df3 = df3.sort_values('Tanggal').reset_index(drop=True)

lags = [1, 3, 7]

lag_columns = [
    "Curah Hujan (mm)",
    "Kelembapan Rata-rata",
    "Temperatur Rata-rata",
    "Temperatur Maksimum",
    "Temperatur Minimum",
    "Lamanya Penyinaran Matahari",
    "Kecepatan Angin Rata-rata"
]

for col in lag_columns:
    for lag in lags:
        df3[f"{col}_lag_{lag}"] = df3[col].shift(lag)

df3["Rain_roll_3"]  = df3["Curah Hujan (mm)"].rolling(3).mean()
df3["Rain_roll_7"]  = df3["Curah Hujan (mm)"].rolling(7).mean()

df3["Humidity_roll_3"] = df3["Kelembapan Rata-rata"].rolling(3).mean()
df3["Humidity_roll_7"] = df3["Kelembapan Rata-rata"].rolling(7).mean()

df3['Rain_ewm_3'] = df3['Curah Hujan (mm)'].ewm(span=3).mean()
df3['Rain_ewm_7'] = df3['Curah Hujan (mm)'].ewm(span=7).mean()

df3['Rain_roll_std_7'] = df3['Curah Hujan (mm)'].rolling(7).std()
df3['Temp_roll_std_7'] = df3['Temperatur Rata-rata'].rolling(7).std()

df3_fe = df3.dropna().reset_index(drop=True)

df3_fe.to_excel("df3_feature_engineered_lag137.xlsx", index=False)


In [24]:
data = pd.read_excel('df3_feature_engineered_lag137.xlsx')

In [26]:
# 1. Pastikan hanya mengambil kolom numerik
numeric_df2 = data.select_dtypes(include=[np.number])

# 2. Hitung korelasi terhadap kolom target saja
target_corr = numeric_df2.corr()['Curah Hujan (mm)'].sort_values(ascending=False)

print(target_corr)

Curah Hujan (mm)                      1.000000
Rain_ewm_3                            0.872986
Rain_ewm_7                            0.685017
Rain_roll_3                           0.657041
Rain_roll_7                           0.470400
Rain_roll_std_7                       0.442558
Kelembapan Rata-rata                  0.397834
Humidity_roll_3                       0.377999
Kelembapan Rata-rata_lag_1            0.366699
Humidity_roll_7                       0.310400
Kelembapan Rata-rata_lag_3            0.191844
Curah Hujan (mm)_lag_1                0.179794
Temp_roll_std_7                       0.164846
Kelembapan Rata-rata_lag_7            0.147756
Curah Hujan (mm)_lag_3                0.092645
Curah Hujan (mm)_lag_7                0.085169
Arah Angin Saat Kecepatan Maksimum    0.082950
Kecepatan Angin Rata-rata_lag_7       0.038841
Kecepatan Angin Rata-rata_lag_3       0.018885
Arah Angin Terbanyak (°)              0.011648
Kecepatan Angin Rata-rata_lag_1       0.007292
Kecepatan Ang

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4013 entries, 0 to 4012
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Tanggal                             4013 non-null   datetime64[ns]
 1   Temperatur Minimum                  4013 non-null   float64       
 2   Temperatur Maksimum                 4013 non-null   float64       
 3   Temperatur Rata-rata                4013 non-null   float64       
 4   Kelembapan Rata-rata                4013 non-null   int64         
 5   Curah Hujan (mm)                    4013 non-null   float64       
 6   Lamanya Penyinaran Matahari         4013 non-null   float64       
 7   Kecepatan Angin Maksimum            4013 non-null   int64         
 8   Arah Angin Saat Kecepatan Maksimum  4013 non-null   int64         
 9   Kecepatan Angin Rata-rata           4013 non-null   int64         
 10  Arah Angin Terbanyak (°)

# Data Modelling 

In [14]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor

df4 = pd.read_excel("df3_feature_engineered_lag137.xlsx")

df4["Rain_Event"] = (df4["Curah Hujan (mm)"] > 0).astype(int)

feature_cols = df4.columns.drop([
    "Tanggal",
    "Curah Hujan (mm)",
    "Rain_Event"
])

X = df4[feature_cols]
y_cls = df4["Rain_Event"]

tscv = TimeSeriesSplit(n_splits=5)
train_idx, test_idx = list(tscv.split(X))[-1]

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train_cls, y_test_cls = y_cls.iloc[train_idx], y_cls.iloc[test_idx]

clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

clf.fit(X_train, y_train_cls)

y_pred_prob = clf.predict_proba(X_test)[:, 1]

rain_threshold = 0.3
y_pred_cls = (y_pred_prob >= rain_threshold).astype(int)

print(
    "Stage 1 — Rain Event Accuracy:",
    accuracy_score(y_test_cls, y_pred_cls)
)

df4_rain = df4[df4["Curah Hujan (mm)"] > 0].copy()

X_rain = df4_rain[feature_cols]
y_rain = np.log1p(df4_rain["Curah Hujan (mm)"])

train_idx_r, test_idx_r = list(tscv.split(X_rain))[-1]

X_train_r, X_test_r = X_rain.iloc[train_idx_r], X_rain.iloc[test_idx_r]
y_train_r, y_test_r = y_rain.iloc[train_idx_r], y_rain.iloc[test_idx_r]

reg = XGBRegressor(
    n_estimators=800,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    reg_alpha=0.5,
    reg_lambda=1.5,
    objective="reg:squarederror",
    random_state=42
)

reg.fit(X_train_r, y_train_r)
final_pred = []

for i in range(len(X_test)):
    prob = y_pred_prob[i]

    if prob < rain_threshold:
        final_pred.append(0)
    else:
        rain_log = reg.predict(X_test.iloc[[i]])[0]
        rain_mm = np.expm1(rain_log)
        final_pred.append(prob * rain_mm)

final_pred = np.array(final_pred)

y_true = df4.iloc[test_idx]["Curah Hujan (mm)"]

# print(
#     "FINAL MAE (Two-Stage):",
#     mean_absolute_error(y_true, final_pred)
# )

mae = mean_absolute_error(y_true, final_pred)
mse = mean_squared_error(y_true, final_pred)
rmse = np.sqrt(mse)

print("FINAL MAE  (Two-Stage):", mae)
print("FINAL MSE  (Two-Stage):", mse)
print("FINAL RMSE (Two-Stage):", rmse)


result = df4.iloc[test_idx][["Tanggal"]].copy()
result["Actual_Rain_mm"] = y_true.values
result["Predicted_Rain_mm"] = final_pred

result.to_excel("two_stage_prediction_result.xlsx", index=False)
print("Selesai! Hasil tersimpan.")


Stage 1 — Rain Event Accuracy: 0.75
FINAL MAE  (Two-Stage): 1.5403436866374831
FINAL MSE  (Two-Stage): 12.188318103449882
FINAL RMSE (Two-Stage): 3.4911771801857725
Selesai! Hasil tersimpan.
